# COGS 108 - Final Project 

*Fill in your overview here*

# Names

- Mariam Issa
- Andrea Sudharta
- Payam Sadeghian
- Brandon Amaral
- Alex Luo
- Jun Wang

# Group Members IDs

- A12285140
- A14497101
- A13654507
- A########
- A########
- A15743932

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis

*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [2]:
import pandas as pd
import tweepy
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import requests
import time

In [ ]:
tweeters_dem_df = pd.read_csv('popular-tweeters-dem.csv') #Dataframe for list of Democratic Party politicians
tweeters_rep_df = pd.read_csv('popular-tweeters-rep.csv') #Dataframe for list of Republican Party politicians
tweeters_dem_df['Party'] = 'D'
tweeters_rep_df['Party'] = 'R'
tweeters_df = pd.concat([tweeters_dem_df, tweeters_rep_df], ignore_index = True) # Dataframe for combined list of politicians

tweeters_handle = list(tweeters_df['Twitter_Handle']) #Python list for politician Twitter handles

In [ ]:
# Get OAuth from Twitter
import tweepy

consumer_key = 'bSZRBubFkHWewMi08ltv7DgAu'
consumer_secret = '6JFpZu66GT7OvoJFFxSfJJlX21NH5wxmMjNDbsQGPLyW9WtBSU'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

try:
    redirect_url = auth.get_authorization_url()
    print (redirect_url)
    
except tweepy.TweepError:
    print("Error! Failed to get request token.")

In [ ]:
# Verify Twitter account usage
verifier = raw_input('Verifier:')

In [ ]:
# Get Access Token
try: 
    auth.get_access_token(verifier)
except tweepy.TweepError:
    print ("Error! Failed to get access token.")

In [ ]:
# Save tokens
access_token = auth.access_token
acess_token_secret = auth.access_token_secret

# Note: You do not need to re-fetch it each time. Twitter currently does
# not expire the tokens, so the only time it would ever go invalid is if
# the user revokes our application access. 

#To rebuild an OAuthHandler from the stored access token

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(key, secret)

In [ ]:
# Check Set Up
api = tweepy.API(auth)
#api.update_status('tweepy + oauth!') # Posts directly on my timeline

In [ ]:
# Filter tweets to only include those containing key words
key_words_by_bill = {
    0:['#savetheinternet', '#netneutrality', 'internet', 'neutrality'], 
    1:['#medicareforall', '#universalhealthcare','#healthcare', 'health', 'care', 'health', 'care', 'medicare'], 
    2: ['#equalpay', '#equalrights', '#equalrightsammendment', '#era', '#women', '#genderinequality', '#genderequality', 'equal', 'rights', 'pay', 'women', 'gender', 'equality'],
    3:['#votingrights', '#votingrightact', '#electionintegrity', '#riggedelections', '#forthepeople', 'voting', 'rights', 'elections', 'fair', 'vote', 'votes'], 
    4:['#guncontrol', '#2ndammendment', '#noguncontol', '#gunviolence', '#gunrights', '#gunreform', '#2a', 'gun', 'second', '2nd',  'amendment'], 
    5:['#yemen', '#yemengenocide', 'yemen', 'yemeni', 'war', 'troops', 'withdraw troops', 'civil'], 
    6:['#women', '#reauthorizationact', '#violenceagainstwomen', '#genderbasedviolence', '#sexualviolence', 'women'],  
    7:['#buildthewall', '#buildthewallnow', '#nationalsecurity', '#nationalemergency', '#trumpswall', '#illegalimmigration', '#mexico', '#border', '#bordercrises', 'mexico', 'illegal', 'immigration', 'security', 'emergency', 'build', 'wall']
}

key_words = list({x for v in key_words_by_bill.itervalues() for x in v})

In [ ]:
# Define Intersection Function
def interSection(arr1,arr2): 
    return list(filter(lambda x: x in arr1, arr2)) 

In [ ]:
# Collect all tweets that deal with each bill in Congress
bill = [{}] * 8 # bill[bill_number] = {congressman: [tweets about bill]}
tweets = {}

In [ ]:
bill_title = {
    0: "Save the Internet",
    1: "Health Care",
    2: "Paycheck Fairness Act",
    3: "For the People Act 2019",
    4: "Background Check for Firearms",
    5: "Remove Troops from Yemen",
    6: "Violence against Women Reauthorization Act",
    7: "Trump's National Emergency"
}

In [ ]:
#Iterate through Congressmen
for handle in tweeters_handle:
    all_tweets = []

    for i in range(1000):
        try:
            tweets_from_page_i = api.user_timeline(handle, count = 200, page = i) # max count value is 200
        except:
            print("TweepError: stopped at handle: ", handle, " at page ", i)
        
        # Check if we reached end of Twitter history
        if len(tweets_from_page_i) == 0: 
            break
            
        #iterate through each tweet, sort by bill
        for tweet in tweets_from_page_i:
            tweet_tokens = (tweet.text).lower().split()
            
            # See if Tweet talks about bill matter
            for bill_number, bill_tokens in key_words_by_bill.items():
                if len(interSection(tweet_tokens, bill_tokens)) != 0:
                    if bill[bill_number] == {}:
                        bill[bill_number] = {handle:[tweet.text]}
                    else:
                        if handle not in bill[bill_number]:
                            bill[bill_number][handle] = [tweet.text]
                        else:
                            bill[bill_number][handle] += [tweet.text]
                    
                    # Compile to all tweets list
                    all_tweets.append(tweet.text)
                    
    # Collect congressman/woman's tweets         
    tweets[handle] = all_tweets
        
    # Sleep: to not overload Twitter requests
    time.sleep(45)

In [ ]:
#Create look up table for Congressman/woman's name and party
tweeters_party = list(tweeters_df['Party'])
tweeters_name = list(tweeters_df['Name'])

handle_to_party = {}
handle_to_name = {}
for i, handle in enumerate(tweeters_handle):
    handle_to_party[handle] = tweeters_party[i]
    handle_to_name[handle] = tweeters_name[i]

data = [] 

# List all tweets by bill it's associated with and the congressman who tweeted it
for i in range(len(bill)):
    for handle, tweet in bill[i].items():
        data.append([handle_to_name[handle], handle_to_party[handle], handle, bill_title[i], tweet])
        
tweeters_df = pd.DataFrame(data, columns = ['Name', 'Party', 'Twitter Handle', 'Bill #', 'Tweet'])

# Export CSV with Tweets
tweeters_df.to_csv('tweets_by_bill')

# Data Cleaning

In [11]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/andrea/OneDrive/Documents/git/truther-tweet//politician-tweet-sentiment-4910a807f774.json"
# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(content=text, type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

ImportError: No module named cloud

In [8]:
# pass in a string
def sentimentAnalysis(text):
    document = types.Document(content=text, type=enums.Document.Type.PLAIN_TEXT)
    
    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    
    return sentiment.score, sentiment.magnitude
    

In [20]:
from nltk.tokenize import word_tokenize

tweets_df = pd.read_csv('tweets_by_bill')

all_sentiment_scores = []
avg_scores = []
mdn_scores = []
rng_scores = [] 
var_scores = [] 
std_dev_scores = []

for bill, tweets in zip(tweets_df['Bill #'], tweets_df['Tweet']):
    
    tweet_scores = []
    
    for twt in tweets.split('u\''): 
        # Clean tweet
        twt = ' '.join([word.lower() for word in twt.split(' ') if word.isalpha()])
        
        # Skip empty strings
        if twt == '': continue 
            
        # Evaluate by bill
        tweet_scores.append(bill_sentiment(twt, bill))
        
    # Add list of scores to Dataframe
    all_sentiment_scores.append(tweet_scores)
    
    # Get AVERAGE tweet sentiment score
    avg = sum(tweet_scores) * 1.0 / len(tweet_scores)
    avg_scores.append(avg)
        
    # Get MEDIAN tweet sentiment score
    sorted_tweet_scores = sorted(tweet_scores)
    mdn = sorted_tweet_scores[len(tweet_scores) / 2]
    mdn_scores.append(mdn)
        
    # Get RANGE tweet sentiment score
    rng = sorted_tweet_scores[len(tweet_scores) - 1] - sorted_tweet_scores[0]
    rng_scores.append(rng)
        
    # Get VARIANCE tweet sentiment score
    var = sum([(score - avg)**2 for score in tweet_scores]) / (len(tweet_scores) - 1)
    var_scores.append(var)
    
    # Get STD DEVIATION tweet sentiment score
    std_dev = var**(1/2)
    std_dev_scores.append(std_dev)
         
# Add to dataframe
tweets_df['Sentiment Scores'] = all_sentiment_scores
tweets_df['Average'] = avg_scores
tweets_df['Median'] = mdn_scores
tweets_df['Range'] = rng_scores
tweets_df['Variance'] = var_scores
tweets_df['Standard Deviation'] = std_dev_scores

# Export new file
tweets_df.to_csv('tweet_sentiment_analysis')

NameError: global name 'types' is not defined

In [21]:
#Note: 1 >= score > 0    Vote YES on bill
#      0 >= score >= -1  Vote NO on bill
def bill_sentiment(tweet, bill):
    if bill == "Save the Internet":
        if ('#savetheinternet' in tweet) or ('#netneutrality' in tweet) or ('neutrality' in tweet):
            return 1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
               
    elif bill == "Health Care":
        if ('#medicareforall' in tweet) or ('#universalhealthcare' in tweet):
            return 1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
        
    elif bill == "Paycheck Fairness Act":
        if 'genderinequality' in tweet:
            return 1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
        
    elif bill == "For the People Act 2019":
        if ('#votingrights' in tweet) or ('#riggedelections' in tweet) or ('#forthepeople' in tweet):
            return 1.0
   
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
        
    elif bill == "Background Check for Firearms":
        if ('#guncontrol' in tweet) or ('#gunviolence' in tweet) or ('gunreform' in tweet):
            return 1.0
        if ('#gunrights' in tweet) or  ('noguncontrol' in tweet):
            return -1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return score
        
    
    elif bill == "Remove Troops from Yemen":
        if '#yemengenocide' in tweet:
            return 1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
    
    elif bill == "Violence against Women Reauthorization Act":
        if ('#violenceagainstwomen' in tweet) or ('#genderbasedviolence' in tweet) or ('#sexualviolence' in tweet):
            return 1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
    
    elif bill == "Trump's National Emergency":
        if  '#trumpswall' in tweet:
            return 1.0
        
        if ('#buildthewall' in tweet) or ('#buildthewallnow' in tweet) or ('#bordercrises' in tweet) or ('#illegalimmigration' in tweet) or ('illegal' in tweet):
            return -1.0
        
        score, mag = sentimentAnalysis(tweet)
        
        return -1 * score
              
    else:
        return 0
        

Describe your data cleaning steps here.

In [22]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*